In [10]:
# input n and T
# The gird n*n and run T times 
# run T the number run experiment
# p the probability of open the site


In [11]:
class UnionFind():
    def __init__(self, n):
        self.parent = list(range(n)) # init parent from 0 to n 
        
    def find(self, x):
        while self.parent[x] != x:
            x = self.parent[x]
        return x
    def union(self, i,j):
        self.parent[i] = self.find(j)

In [12]:
# Initialize the UnionFind data structure with 10 elements
n = 10
uf = UnionFind(n)

# Perform some union operations
uf.union(1, 2)
uf.union(2, 4)
# Perform another union operation
uf.union(5, 6)
uf.union(4, 5)

In [17]:
# check some data
print("find of 2", uf.find(2))
print("find of 5", uf.find(5))
print(uf.parent)

find of 2 6
find of 5 6
[0, 2, 4, 3, 6, 6, 6, 7, 8, 9]
